In [ ]:
# default_exp core

# Core

> Core tools for working with storage.

In [ ]:
#export
from abc import ABC,abstractmethod
from configparser import ConfigParser
from pathlib import Path
import shutil, boto3 as aws, azure.storage.blob as az

In [ ]:
from fastcore.test import *
from configparser import SectionProxy

In [ ]:
#export
def read_config(section_name=None,config_name='secrets//settings.ini'):
    config_path=Path(config_name)
    config=ConfigParser()
    config.read(config_path)
    if section_name is None:
        return config
    if section_name not in config:
        raise Exception(f'Error: [{section_name}] section not found in {config_path}')
    return dict(config.items(section_name))

In [ ]:
assert isinstance(read_config(),ConfigParser)
assert isinstance(read_config()['DEFAULT'],SectionProxy)
assert isinstance(read_config('DEFAULT'),dict)
assert read_config('DEFAULT')['local_path']=='data'
assert read_config('local_cwd',config_name='test//secrets//settings.ini')['storage_type']=='local'

In [ ]:
#export
class StorageClientABC(ABC):
    """Defines functionality common to all storage clients"""
    
    def __init__(self,storage_name,config_name='secrets//settings.ini'):
        "Create a new storage client using the `storage_name` section of `config_name`"
        self.config=read_config(storage_name,config_name=config_name)

    @abstractmethod
    def ls(self,what):
        "Return a list containing the names of files in either `storage_area` or `local_path`"
        
    @abstractmethod
    def download(self,filename): 
        "Copy `filename` from `storage_area` to `local_path`"
    
    @abstractmethod
    def upload(self,filename,overwrite=False): 
        "Copy `filename` from `local_path` to `storage_area`"

In [ ]:
#export
class LocalStorageClient(StorageClientABC):
    """Storage client that uses the local filesystem for both `storage_area` and `local_path`"""
    
    def _ls(self,p,result,len_path_prefix):
        for _p in p.iterdir():
            if _p.is_dir(): self._ls(_p,result,len_path_prefix)
            else: result.append(str(_p).replace('\\','/')[len_path_prefix:])

    def ls(self,what='storage_area'):
        result,p=[],Path(self.config[what])
        p.mkdir(parents=True,exist_ok=True)
        self._ls(p,result,len(self.config[what])+1)
        sorted(result)
        return result
        
    def _cp(self,from_key,to_key,filename,overwrite=False):
        src=Path(self.config[from_key])/filename
        dst=Path(self.config[to_key])/filename
        if dst.exists() and not overwrite: 
            raise FileExistsError(f'{dst} exists and overwrite=False')
        dst.parent.mkdir(parents=True,exist_ok=True)
        shutil.copy(src,dst)
        
    def download(self,filename,overwrite=False):
        try: self._cp('storage_area','local_path',filename,overwrite)
        except FileExistsError: pass
        
    def upload(self,filename,overwrite=False): 
        self._cp('local_path','storage_area',filename,overwrite)

`LocalStorageClient` will most often be used for local testing.

In [ ]:
storage_client=LocalStorageClient('local_test','test//secrets//settings.ini')
assert storage_client.config['storage_type']=='local'

In [ ]:
#export
class AzureStorageClient(StorageClientABC):
    """Storage client that uses Azure for `storage_area` and the local filesystem `local_path`"""
    def _client(self):
        if getattr(self,'client') is None:
            service_client=az.BlobServiceClient.from_connection_string(
                self.config['conn_str'],self.config['credential'])
            self.client=service_client.get_container_client(self.config['container'])
        return self.client
    def ls(self): pass 
    def download(self,filename): pass 
    def upload(self,filename,overwrite=False): pass

In [ ]:
#export
class AwsStorageClient(StorageClientABC):
    """Storage client that uses AWS for `storage_area` and the local filesystem `local_path`"""
    def ls(self): pass 
    def download(self,filename): pass 
    def upload(self,filename,overwrite=False): pass

In [ ]:
#export
def new_storage_client(storage_name,config_name='secrets//settings.ini'):
    "Returns a storage client based on the configured `storage_type`"
    config=read_config(storage_name,config_name=config_name)
    storage_type=config['storage_type']
    if storage_type=='local': return LocalStorageClient(storage_name, config_name)
    elif storage_type=='azure': return AzureStorageClient(storage_name, config_name)
    elif storage_type=='aws': return AwsStorageClient(storage_name, config_name)
    else: raise ValueError(f'Unknown storage_type: {storage_type}')

In [ ]:
test_fail(lambda: new_storage_client('gcp_dummy','test//secrets//settings.ini'))

In [ ]:
def _rmtree(p):
    try: shutil.rmtree(p)
    except FileNotFoundError: pass

In [ ]:
for p in ['test/local_path','test/storage_area']: _rmtree(p)
    
storage_client=new_storage_client('local_test','test//secrets//settings.ini')
assert isinstance(storage_client,LocalStorageClient)
assert storage_client.config['storage_type']=='local'
test_eq([],storage_client.ls())
test_eq([],storage_client.ls('local_path'))
    
test_files=['a/b/test_data2.txt','sub/test_data1.txt','test_data.txt']
for i,f in enumerate(test_files):
    f='test/local_path/'+f
    Path(f).parent.mkdir(parents=True,exist_ok=True)
    with open(f, 'w') as _file: _file.write(f'a little bit of data {i}')
test_eq([],storage_client.ls())
test_eq(test_files,storage_client.ls('local_path'))
        
for f in test_files: storage_client.upload(f)
test_eq(test_files,storage_client.ls())
test_eq(test_files,storage_client.ls('local_path'))
_rmtree('test/local_path')
test_eq([],storage_client.ls('local_path'))

for f in test_files: storage_client.download(f)
test_eq(test_files,storage_client.ls('local_path'))
test_eq('a little bit of data 2',open('test/local_path/test_data.txt').read())

with open('test/local_path/test_data.txt', 'w') as _file: _file.write('upd')
test_eq('upd',open('test/local_path/test_data.txt').read())
storage_client.download('test_data.txt')
test_eq('upd',open('test/local_path/test_data.txt').read())
storage_client.download('test_data.txt',True)
test_eq('a little bit of data 2',open('test/local_path/test_data.txt').read())

test_fail(lambda: storage_client.upload('test_data.txt'))
storage_client.upload('test_data.txt',True)

In [ ]:
storage_client=new_storage_client('azure_dummy','test//secrets//settings.ini')
assert isinstance(storage_client,AzureStorageClient)
storage_client=new_storage_client('aws_dummy','test//secrets//settings.ini')
assert isinstance(storage_client,AwsStorageClient)